In [1]:
import tensorflow.compat.v1 as tf
import pandas as pd

In [2]:
print(tf.__version__)

1.15.0


In [3]:
import sys
print(sys.executable)

/mlx_devbox/users/zhangliang.thanks/playground/conda/envs/py36_bytefed/bin/python


In [4]:
!where python

/bin/sh: 1: where: not found


In [5]:
print(tf.test.is_gpu_available())

False


In [6]:
def train_test_split_fed(df_y, df_x, train_count = 10000):
    y_train = df_y[0:train_count]
    y_test = df_y[train_count+1:]
    x_train = df_x[0:train_count]
    x_test = df_x[train_count+1:]

    return y_train, y_test, x_train, x_test

In [7]:
df = pd.read_csv('data/insurance_claims_label_feature_fed.csv')

print('df shape: {}'.format(df.shape))
df = df[df['ClaimAmount'] > 0]
df.head()

df shape: (24944, 94)


,raw_id,example_id,event_date,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,...,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74
0,1,1,20240901,1.0,0.75,F,7.0,1.0,61.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,50.0,2.257113
1,2,2,20240901,1.0,0.14,B,12.0,5.0,50.0,60.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,60.0,-1.045961
2,3,3,20240901,1.0,0.14,E,4.0,0.0,36.0,85.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,85.0,1.332797
3,4,4,20240901,2.0,0.62,F,10.0,0.0,51.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0,2.257113
4,5,5,20240901,1.0,0.31,A,5.0,0.0,45.0,50.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,50.0,-1.869529


In [8]:
columns_x = ['X{}'.format(idx) for idx in range(75)]
df_x = df[columns_x]

columns_y = ['PurePremium', 'Exposure']
df_y = df[columns_y]

print('df_x shape: {}'.format(df_x.shape))
df_x.head()

df_x shape: (24944, 75)


,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,50.0,2.257113
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,60.0,-1.045961
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,85.0,1.332797
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0,2.257113
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,50.0,-1.869529


In [9]:
# from sklearn.model_selection import train_test_split

# y_train, y_test, x_train, x_test = train_test_split(df_y, df_x, random_state=0)
y_train, y_test, x_train, x_test = train_test_split_fed(df_y, df_x, train_count=20000)

print('y_train shape: {}, \t y_test shape: {}, \n x_train shape: {}, \t x_test shape: {}'
      .format(y_train.shape, y_test.shape, x_train.shape, x_test.shape))

y_train shape: (20000, 2), 	 y_test shape: (4943, 2), 
 x_train shape: (20000, 75), 	 x_test shape: (4943, 75)


In [10]:
########################################################################
# 定义模型的网络结构
########################################################################

# 转换为张量
x_tensor = tf.constant(x_train, dtype=tf.float32)
y_tensor = tf.constant(y_train['PurePremium'], dtype=tf.float32)

# 定义模型参数
# weights = tf.Variable(tf.random.normal([75, 1]), dtype=tf.float32, initializer=tf.random_uniform_initializer(-0.01, 0.01))
# bias = tf.Variable(tf.random.normal([1]), dtype=tf.float32, initializer=tf.zeros_initializer())

# weights = tf.get_variable('weights', shape=[75, 1], dtype=tf.float32, initializer=tf.random_uniform_initializer(-0.01, 0.01))
weights = tf.get_variable('weights', shape=[75, 1], dtype=tf.float32, initializer=tf.zeros_initializer())
bias = tf.get_variable('bias', shape=[1], dtype=tf.float32, initializer=tf.zeros_initializer())

# 定义Tweedie回归模型
def tweedie_regression_exp(X):
    logits = tf.matmul(X, weights) + bias
    return tf.exp(logits)  # 使用指数函数确保输出为正值

# 定义Tweedie损失函数
def tweedie_loss_func(y_true, y_pred, p=1.5):
    '''
        - https://towardsdatascience.com/tweedie-loss-function-for-right-skewed-data-2c5ca470678f
    '''
    loss = - y_true * tf.pow(y_pred, 1 - p) / (1 - p) + tf.pow(y_pred, 2 - p) / (2 - p)
    return tf.reduce_mean(loss)

# 定义优化器
# optimizer = tf.optimizers.Adam(learning_rate=0.01)
optimizer = tf.train.FtrlOptimizer(learning_rate=0.5,
                                            l1_regularization_strength=1e-05,
                                            l2_regularization_strength=0.0005945795938393141,
                                            initial_accumulator_value=0.44352,
                                            learning_rate_power=-0.59496)

# 训练模型
num_epochs = 5000
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        y_pred = tweedie_regression_exp(x_tensor)
        loss = tweedie_loss_func(y_tensor, y_pred)
        
    gradients = tape.gradient(loss, [weights, bias])
    optimizer.apply_gradients(zip(gradients, [weights, bias]))

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

    if loss <= 50:
        break

# 进行预测
pred_tf = tweedie_regression_exp(x_tensor)
print(pred_tf[:5].numpy())


AttributeError: 'RefVariable' object has no attribute '_id'

In [ ]:
# tf预测值
pred_tf = tweedie_regression_exp(x_tensor)
print(pred_tf)

tf.Tensor(
[[7236.9507]
 [9113.418 ]
 [9614.853 ]
 ...
 [8007.618 ]
 [7858.693 ]
 [9271.069 ]], shape=(10000, 1), dtype=float32)


In [ ]:
y_train

,PurePremium,Exposure
0,404.000000,0.75
1,14156.000000,0.14
2,10403.928571,0.14
3,17474.193548,0.62
4,12860.225806,0.31
...,...,...
9995,1446.913580,0.81
9996,2547.826087,0.46
9997,1465.000000,0.80
9998,3084.210526,0.38


In [ ]:
y_test_pred = tweedie_regression_exp(tf.constant(x_test, dtype=tf.float32))
y_train_pred = tweedie_regression_exp(tf.constant(x_train, dtype=tf.float32))

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_train['PurePremium'], y_train_pred, sample_weight=y_train['Exposure'])
print("Train Mean Squared Error:", mse)

mse = mean_squared_error(y_test['PurePremium'], y_test_pred, sample_weight=y_test['Exposure'])

print("Test Mean Squared Error:", mse)

Train Mean Squared Error: 753601669.9884224
Test Mean Squared Error: 661635987.0438185


In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_train['PurePremium'], y_train_pred, sample_weight=y_train['Exposure'])
print("Train Mean A Error:", mae)

mae = mean_absolute_error(y_test['PurePremium'], y_test_pred, sample_weight=y_test['Exposure'])

print("Test Mean A Error:", mae)

Train Mean A Error: 7441.417514946629
Test Mean A Error: 7565.239419459457
